In [5]:
import matplotlib.pyplot as plt
import numpy as np
import yfinance as yf
from backtesting import Backtest, Strategy







In [44]:
import numpy as np
import pandas as pd
import os
from backtesting import Backtest, Strategy
from datetime import datetime, timedelta, date
import yfinance as yf
from tqdm import tqdm
import pytz
import talib


def walk_forward_macd(df_prices, MACDStrategy, iterations, fast_periods, slow_periods, signal_periods, metric, output_file='macd_walk_forward_results.csv'):
    report = []

    # Iterate over the list of iterations
    for iter in tqdm(iterations):
        # Filter the data to only include the relevant dates
        df_is = df_prices[(df_prices.index >= iter['in_sample'][0]) & (df_prices.index <= iter['in_sample'][1])]
        df_oos = df_prices[(df_prices.index >= iter['out_of_sample'][0]) & (df_prices.index <= iter['out_of_sample'][1])]

        # Calculate the optimal parameters using the in-sample data
        stats_is, heatmap = optimize_macd_strategy(
            df_is, MACDStrategy, fast_periods, slow_periods, signal_periods, metric
        )

        # Run the backtest for the out-of-sample data using the optimal parameters
        fast_period = stats_is._strategy.fast_period
        slow_period = stats_is._strategy.slow_period
        signal_period = stats_is._strategy.signal_period

        stats_oos = execute_macd_strategy(
            df_oos, MACDStrategy, fast_period=fast_period, slow_period=slow_period, signal_period=signal_period
        )

        def calculate_wfe(oos, ins):
            if ins != 0:
                return (oos/ins)

        wfe = calculate_wfe(stats_oos['Sharpe Ratio'], stats_is['Sharpe Ratio'])

        # Append relevant metrics to a list of results
        report.append({
            'start_date': stats_oos['Start'],
            'end_date': stats_oos['End'],
            'return_strat': stats_oos['Return [%]'],
            'max_drawdown': stats_oos['Max. Drawdown [%]'],
            'ret_strat_ann': stats_oos['Return (Ann.) [%]'],
            'profit_factor': stats_oos['Profit Factor'],
            'volatility_strat_ann': stats_oos['Volatility (Ann.) [%]'],
            'is_sharpe_ratio': stats_is['Sharpe Ratio'],
            'oos_sharpe_ratio': stats_oos['Sharpe Ratio'],
            'return_bh': stats_oos['Buy & Hold Return [%]'],
            'WFE': wfe,
            'fast_period': fast_period,
            'slow_period': slow_period,
            'signal_period': signal_period
        })

    df_report = pd.DataFrame(report)
    df_report.to_csv(output_file, index=False)


if __name__ == "__main__":
    # List of currency tickers
    tickers = ["EURUSD=X", "GBPUSD=X", "JPY=X", "AUDUSD=X", "CAD=X", "CHF=X", "NZDUSD=X", "CNY=X", "HKD=X", "SGD=X", "INR=X", "RUB=X", "MXN=X", "ZAR=X", "KRW=X"]

    start_date = "2023-01-01"
    end_date = "2024-11-22"

    # Assuming getYahooFinanceData() fetches historical data from Yahoo Finance
    data_obj = getYahooFinanceData()
    currency_data = data_obj.fetch_data(start_date, end_date, tickers)

    # Define strategy parameters
    metric = "Sharpe Ratio"
    fast_periods = range(5, 30, 1)  # Fast EMA periods
    slow_periods = range(20, 50, 1)  # Slow EMA periods
    signal_periods = range(5, 20, 1)  # Signal line periods
    cash = 10**5
    commission = 0.001
    start_date = date(2023, 1, 1)
    end_date = datetime(2024, 11, 22)

    # Define the parameters of the iterations (e.g., 6 months in-sample and 6 months out-of-sample)
    in_sample_duration = pd.DateOffset(months=6)
    out_of_sample_duration = pd.DateOffset(months=6)

    iterations = []
    for i in range(0, 3):
        in_sample_start = start_date + i * in_sample_duration
        in_sample_end = in_sample_start + in_sample_duration - timedelta(hours=1)
        out_of_sample_start = in_sample_end + timedelta(hours=1)
        out_of_sample_end = out_of_sample_start + out_of_sample_duration - timedelta(hours=1)

        # Make the datetime objects timezone-aware in UTC
        in_sample_start = pytz.utc.localize(in_sample_start)
        in_sample_end = pytz.utc.localize(in_sample_end)
        out_of_sample_start = pytz.utc.localize(out_of_sample_start)
        out_of_sample_end = pytz.utc.localize(out_of_sample_end)

        iterations.append({
            'in_sample': [in_sample_start, in_sample_end],
            'out_of_sample': [out_of_sample_start, out_of_sample_end]
        })

    for currency, data in currency_data.items():
        if data.index.tz is None:
            # Localize to UTC
           
            data.index = data.index.tz_localize('UTC')

        data.index = data.index.tz_convert('UTC')
        print(len(data))

        output_file = 'output/walk_forward_results_' + currency + '.csv'

        walk_forward_macd(data, MACDStrategy, iterations, fast_periods, slow_periods, signal_periods, metric, output_file=output_file)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

object initialized
data pull complete for:  EURUSD=X
data pull complete for:  GBPUSD=X
data pull complete for:  JPY=X



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


data pull complete for:  AUDUSD=X
data pull complete for:  CAD=X
data pull complete for:  CHF=X
data pull complete for:  NZDUSD=X
data pull complete for:  CNY=X
data pull complete for:  HKD=X
data pull complete for:  SGD=X
data pull complete for:  INR=X


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


...

 33%|███████████████                              | 1/3 [00:01<00:03,  1.85s/it]/opt/anaconda3/lib/python3.12/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/9 [00:00<?, ?it/s]

 67%|██████████████████████████████               | 2/3 [00:03<00:01,  1.86s/it]/opt/anaconda3/lib/python3.12/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/9 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████| 3/3 [00:05<00:00,  1.83s/it]


In [ ]:
import numpy as np
import pandas as pd
import os
from backtesting import Backtest, Strategy
from datetime import datetime, timedelta, date
import yfinance as yf
from tqdm import tqdm
import pytz
import talib


def localize_data(data):
  """
  Localizes the index of a DataFrame to UTC

  Args:
      data: DataFrame with potentially non-localized index

  Returns:
      A DataFrame with its index localized to UTC
  """
  if data.index.tz is None:
    data.index = data.index.tz_localize('UTC')
  data.index = data.index.tz_convert('UTC')
  return data


def define_walk_forward_iterations(start_date, end_date, in_sample_duration, out_of_sample_duration, num_iterations):
  """
  Defines a list of dictionaries representing walk-forward iterations

  Args:
      start_date: Overall start date (date object)
      end_date: Overall end date (datetime object)
      in_sample_duration: Length of the in-sample period (DateOffset object)
      out_of_sample_duration: Length of the out-of-sample period (DateOffset object)
      num_iterations: Number of walk-forward iterations (int)

  Returns:
      A list of dictionaries where each dictionary represents an iteration with
      in-sample and out-of-sample start and end dates
  """
  iterations = []
  for i in range(num_iterations):
    in_sample_start = start_date 
    in_sample_end = in_sample_start + in_sample_duration - timedelta(hours=1)
    out_of_sample_start = in_sample_end + timedelta(hours=1)
    out_of_sample_end = out_of_sample_start + out_of_sample_duration - timedelta(hours=1)

    # Make the datetime objects timezone-aware in UTC
    in_sample_start = pytz.utc.localize(in_sample_start)
    in_sample_end = pytz.utc.localize(in_sample_end)
    out_of_sample_start = pytz.utc.localize(out_of_sample_start)
    out_of_sample_end = pytz.utc.localize(out_of_sample_end)

    iterations.append({
        'in_sample': [in_sample_start, in_sample_end],
        'out_of_sample': [out_of_sample_start, out_of_sample_end]
    })
  return iterations


def calculate_walk_forward_metric(oos_metric, ins_metric):
  """
  Calculates a Walk Forward metric (e.g., WFE)

  Args:
      oos_metric: Out-of-sample metric value (float)
      ins_metric: In-sample metric value (float)

  Returns:
      The calculated Walk Forward metric (float)
  """
  if ins_metric != 0:
    return (oos_metric / ins_metric)
  else:
    return np.nan  # Handle division by zero


In [ ]:
"""This script executes, optimizes, or performs a train-test split
optimization on a MACD trading strategy for specified instruments"""

import sys
import logging
import pandas as pd
import matplotlib.pyplot as plt
from utils import utils
from lib_macd import macd_modules as mm
from configs import configs
import seaborn as sns


def execute_macd(ticker, data, fast_period, slow_period, signal_period):
    """
    Execute the MACD strategy and save the results as CSV and plots.

    Args:
        ticker (str): The ticker symbol for the financial instrument.
        data (DataFrame): The market data for the instrument.
        fast_period (int): The fast period for the MACD strategy.
        slow_period (int): The slow period for the MACD strategy.
        signal_period (int): The signal period for the MACD strategy.
    """
    try:
        report = []
        stats = mm.execute_macd_strategy(
            data, mm.MACDStrategy, fast_period=fast_period,
            slow_period=slow_period, signal_period=signal_period)

        report.append({
            'start_date': stats['Start'],
            'end_date': stats['End'],
            'return_strat': stats['Return [%]'],
            'max_drawdown': stats['Max. Drawdown [%]'],
            'ret_strat_ann': stats['Return (Ann.) [%]'],
            'profit_factor': stats['Profit Factor'],
            'volatility_strat_ann': stats['Volatility (Ann.) [%]'],
            'sharpe_ratio': stats['Sharpe Ratio'],
            'return_bh': stats['Buy & Hold Return [%]'],
            'cagr': stats['CAGR (%)'],
            'fast_period': fast_period,
            'slow_period': slow_period,
            'signal_period': signal_period
        })

def optimize_macd(ticker, data, fast_periods, slow_periods, signal_periods):
    """
    Optimize the MACD strategy and save the results as CSV and plots.

    Args:
        ticker (str): The ticker symbol for the financial instrument.
        data (DataFrame): The market data for the instrument.
        fast_periods (range): The range of fast periods to optimize.
        slow_periods (range): The range of slow periods to optimize.
        signal_periods (range): The range of signal periods to optimize.
    """
    try:
        report = []
        metric = "Sharpe Ratio"
        stats, heatmap = mm.optimize_macd_strategy(
            data, mm.MACDStrategy, fast_periods,
            slow_periods, signal_periods, metric)

        report.append({
            'start_date': stats['Start'],
            'end_date': stats['End'],
            'return_strat': stats['Return [%]'],
            'max_drawdown': stats['Max. Drawdown [%]'],
            'ret_strat_ann': stats['Return (Ann.) [%]'],
            'profit_factor': stats['Profit Factor'],
            'volatility_strat_ann': stats['Volatility (Ann.) [%]'],
            'sharpe_ratio': stats['Sharpe Ratio'],
            'return_bh': stats['Buy & Hold Return [%]'],
            'cagr': stats['CAGR (%)'],
            'fast_period': stats._strategy.fast_period,
            'slow_period': stats._strategy.slow_period,
            'signal_period': stats._strategy.signal_period
        })




In [ ]:
walk_forward_results = pd.DataFrame()

for filename in os.listdir('output/'):
    file_details = filename.split('_')
    file_details = [x.replace('.csv', '') for x in file_details]
    
    # Load the walk-forward results
    if file_details[0] == 'walk':
        currency = file_details[3]
        if currency in tickers:
            df_results = pd.read_csv(f"output/walk_forward_results_{currency}.csv")
            df_results['currency'] = currency
            walk_forward_results = pd.concat([df_results, walk_forward_results])

walk_forward_results['start_date'] = walk_forward_results['start_date'].apply(lambda x: datetime.strptime(x[:-6], '%Y-%m-%d %H:%M:%S').date())
walk_forward_results['end_date'] = walk_forward_results['end_date'].apply(lambda x: datetime.strptime(x[:-6], '%Y-%m-%d %H:%M:%S').date())
walk_forward_results = walk_forward_results.rename(columns={'ret_strat_ann':'ret_ann', 'volatility_strat_ann':'vol_ann'})

# walk_forward_results['start_date'] = walk_forward_results['start_date'].apply(lambda x: datetime.strptime(x[:-6], '%Y-%m-%d %H:%M:%S').date())
# walk_forward_results['end_date'] = walk_forward_results['end_date'].apply(lambda x: datetime.strptime(x[:-6], '%Y-%m-%d %H:%M:%S').date())
# walk_forward_results = walk_forward_results.rename(columns={'ret_strat_ann':'ret_ann', 'volatility_strat_ann':'vol_ann'})

agg_list = ['return_strat', 'max_drawdown', 'ret_ann', 'profit_factor', 'vol_ann', 'is_sharpe_ratio', 'oos_sharpe_ratio', 'return_bh', 'WFE', 'fast_period', 'slow_period', 'signal_period']
round(walk_forward_results.groupby(['currency', 'start_date', 'end_date']).agg({x:'mean' for x in agg_list}).fillna(0), 2).to_csv('output/combined_wfo_results.csv')



In [7]:
## MACd on forex

In [1]:
# import numpy as np
# import pandas as pd
# import os
# from backtesting import Backtest, Strategy
# from datetime import datetime, timedelta, date
# import yfinance as yf
# from tqdm import tqdm
# import pytz
# import talib

# tickers = ["EURUSD=X", "GBPUSD=X", "JPY=X", "AUDUSD=X", "CAD=X", "CHF=X", "NZDUSD=X", "CNY=X", "HKD=X", "SGD=X", "INR=X", "RUB=X", "MXN=X", "ZAR=X", "KRW=X"]

# start_date = "2023-01-01"
# end_date = "2024-11-22"

# data_obj = getYahooFinanceData()
# currency_data = data_obj.fetch_data(start_date, end_date, tickers)

# metric = "Sharpe Ratio"
# fast_periods = range(5, 80, 1)
# slow_periods = range(5, 80, 1)
# signal_periods = range(5, 80, 1)
# cash = 10**5

# report = []
# for currency, data in currency_data.items():
#     print("running for currency: ", currency)
#     stats_is, heatmap = optimize_macd_strategy(data, MACDStrategy, fast_periods, slow_periods, signal_periods, metric)
#     report.append({
#             'currency': currency,
#             'return_strat': stats_is['Return [%]'],
#             'max_drawdown': stats_is['Max. Drawdown [%]'],
#             'ret_strat_ann': stats_is['Return (Ann.) [%]'],
#             'profit_factor': stats_is['Profit Factor'],
#             'volatility_strat_ann': stats_is['Volatility (Ann.) [%]'],
#             'is_sharpe_ratio': stats_is['Sharpe Ratio'],
#             'oos_sharpe_ratio': stats_is['Sharpe Ratio'],
#             'return_bh': stats_is['Buy & Hold Return [%]'],
#             'fast_period': stats_is._strategy.fast_period,
#             'slow_period': stats_is._strategy.slow_period,
#             'signal_period': stats_is._strategy.signal_period,
#         })


In [17]:
# pd.DataFrame(report).to_csv("output/currency_results.csv")